In [30]:
import pickle
import pandas as pd
import json 
import numpy as np
import sys 
sys.path.append('../')

from data_manipulation.read_data import get_site_by_id
from data_manipulation.feature_engineering import add_seasonality, clean_weather_features, reorder_columns, features
from model_prediction import generate_prod_prediction, get_weather_forecast_df, get_irr_forecast_df, join_forecast_df, load_prod_model

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
model = load_prod_model(103941)
preds = generate_prod_prediction(103941, model)
preds

Making request to url: https://api.darksky.net/forecast/09af10825ba1cd8092ba13b76c911acd/36.9587533,-112.9758274
Making request to url: https://api.solcast.com.au/world_radiation/forecasts?latitude=36.9587533&longitude=-112.9758274&api_key=DH7r5R0ctJhIyHxnprkKEwjGcQl6cuR9&format=json


date
2020-05-05 21:00:00+00:00    1275.114416
2020-05-05 22:00:00+00:00     977.754603
2020-05-05 23:00:00+00:00     664.470494
2020-05-06 00:00:00+00:00     468.899466
2020-05-06 01:00:00+00:00     514.940597
2020-05-06 02:00:00+00:00     678.743572
2020-05-06 03:00:00+00:00     847.368141
2020-05-06 04:00:00+00:00     761.174193
2020-05-06 05:00:00+00:00     763.628981
2020-05-06 06:00:00+00:00     752.928787
2020-05-06 07:00:00+00:00     754.956446
2020-05-06 08:00:00+00:00     744.246525
2020-05-06 09:00:00+00:00     710.546669
2020-05-06 10:00:00+00:00     709.576299
2020-05-06 11:00:00+00:00     570.883594
2020-05-06 12:00:00+00:00     568.870597
2020-05-06 13:00:00+00:00     646.445093
2020-05-06 14:00:00+00:00     441.477585
2020-05-06 15:00:00+00:00     317.300625
2020-05-06 16:00:00+00:00     472.963580
2020-05-06 17:00:00+00:00     664.522400
2020-05-06 18:00:00+00:00     731.875634
2020-05-06 19:00:00+00:00     991.493414
2020-05-06 20:00:00+00:00     869.286987
2020-05-06 